In [ ]:
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title
from langchain_core.documents import Document


In [ ]:
path = '../../docs'

In [ ]:
import os
path, dirs, files = next(os.walk(path))
files

In [ ]:
documents = []
for file in files:
    with open( os.path.join(path, file), 'rb') as f:
        print(f'Opening file: {file}')
        
        # open file, clean, segment
        elements = partition(file=f)
        
        # create chunks to better rag
        chunks = chunk_by_title(elements, max_characters=1500, new_after_n_chars=1000, overlap=150)
        
        # convert to the Langchain format
        for chunk in chunks:
            metadata = chunk.metadata.to_dict()
            # print("---------------------")
            # print(metadata)
            # print("---------------------")
            # print(chunk.text)

            del metadata["languages"]
            if "filename" in metadata:
                metadata["source"] = metadata["filename"]
            else:
                metadata["source"] = file

            documents.append(Document(page_content=chunk.text, metadata=metadata))


In [ ]:
documents

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

In [ ]:
index_name = "index1"
url = 'host.docker.internal'
url2 = 'http://host.docker.internal'

In [ ]:
import weaviate

weaviate_client = weaviate.connect_to_custom(
    http_host=url,
    http_port=8080,
    http_secure=False,
    grpc_host=url,
    grpc_port=50051,
    grpc_secure=False,
    # auth_credentials=AuthApiKey(weaviate_key),   # `weaviate_key`: your Weaviate API key
)

In [ ]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

vector_store = WeaviateVectorStore.from_documents(documents, embeddings, client=weaviate_client, index_name=index_name)

In [ ]:
vector_store = WeaviateVectorStore(weaviate_client, index_name, "text_key")

In [1]:
import sys
import logging
logging.basicConfig(level=logging.INFO,  # Define o nível de log
                    format='%(asctime)s - %(levelname)s - %(message)s',  # Define o formato da mensagem de log
                    stream=sys.stdout)  # Define a saída do log para stdout


In [2]:
import sys
import os

# Assuming the notebook is in 'app/lab' and the module is in 'app/app'
module_path = os.path.abspath(os.path.join('..', 'app'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from ingest import ingest

ingest.ingest_path("../../docs_2", "index2", logging)

2024-05-20 00:25:20,192 - INFO - Importing files to the vector store
Opening file: Cadastramento inicial no PJe.pdf
2024-05-20 00:25:45,626 - INFO - pikepdf C++ to Python logger bridge initialized
Opening file: guiapje-advogados.pdf
2024-05-20 00:25:56,676 - INFO - HTTP Request: GET http://host.docker.internal:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
2024-05-20 00:25:56,842 - INFO - HTTP Request: GET http://host.docker.internal:8080/v1/meta "HTTP/1.1 200 OK"
2024-05-20 00:25:57,048 - INFO - HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
2024-05-20 00:25:57,328 - INFO - HTTP Request: GET http://host.docker.internal:8080/v1/schema/Index2 "HTTP/1.1 404 Not Found"
2024-05-20 00:25:57,435 - INFO - HTTP Request: POST http://host.docker.internal:8080/v1/schema "HTTP/1.1 200 OK"
2024-05-20 00:25:57,440 - INFO - HTTP Request: GET http://host.docker.internal:8080/v1/schema/Index2 "HTTP/1.1 200 OK"
2024-05-20 00:26:11,644 - INFO - HTTP Requ

'Ingestion succesfull'